In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
points=pd.read_csv('/home/ashwin/Downloads/sec_order_opt.csv')
points.head()

,dist_cycled,calories
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925


In [10]:
def error_caluclation(m,b):
    totalerror=0
    for i in range(len(points)):
        x=points[:,0]
        y=points[:,1]
        totalerror+=(y-(m*x+b))**2
    return totalerror/float(len(points))

In [11]:
def caluclate_the_total_error(point_pair):
    return error_caluclation(point_pair[0],point_pair[1])

In [12]:
def compute_jacobian(point_pair,h=1e-5):
    n=len(point_pair)
    jacobian=np.zeros(n)
    for i in range(n):
        x_i=np.zeros(n)
        x_i[i]+=h
        jacobian[i]=(caluclate_the_total_error(point_pair+x_i)
                     -caluclate_the_total_error(point_pair))/h
    return jacobian

In [12]:
def compute_hassian(point_pair,h=1e-5):
    n=len(point_pair)
    hassian=np.zeros((n,n))
    for i in range(n):
        x_i=np.zeros()
        x_i[i]+=h
        hassian[i]=(compute_jacobian(point_pair+x_i)
                    -compute_jacobian(point_pair))/h
    return hassian

In [14]:
def compute_newton(init_points, max_iter = 10000, e = 1e-5): #calculate roots of the equation, i.e. find x if f(x) = 0. In our case we want to find the minima point, so we find f'(x) = 0
    point_pair_arr = np.zeros((max_iter, len(init_points))) #initalize m,b values
    point_pair_arr[0] = init_points #start points
    opt_val = None #optimal_value to return
    for i in range(max_iter):
        jacobian = compute_jacobian(point_pair_arr[i]) #calculate the jacobian at current m,b
        hessian = compute_hassian(point_pair_arr[i]) #calculate the hessian at current m,b
        point_pair_arr[i+1] = point_pair_arr[i] - np.dot(np.linalg.pinv(hessian), jacobian) #calulate the new m, new b using newton's equation x(t+1) = x(t) - f(x(t))/f'(x(t)) but we want to find root of f'(x) so we would do x(t+1) = x(t) - f'(x(t))/f''(x(t))
        #pinv is pseudo inverse, it prevents values like 1/0 and replaces it with a very high value.
        print('New m is %.2f and new b is %.2f'%(point_pair_arr[i,0], point_pair_arr[i,1]))
        opt_val = point_pair_arr[i+1]
        if np.abs(caluclate_the_total_error(point_pair_arr[i+1]) - caluclate_the_total_error(point_pair_arr[i])) < e: #used for early stopping, stops when there is no real improvement.
            print('Optimal m is %.2f and Optimal b is %.2f'%(point_pair_arr[i+1,0], point_pair_arr[i+1,1]))
            break

    return opt_val

In [15]:
def plot_line_data(m, b): #Plots the calculated line from m and b
    X = points[:,0]
    Y = points[:,1]
    plt.plot(X, Y, 'bo') #First plots the data points
    plt.plot(X, m * X + b) #Plot the line.
    plt.axis([0,1.5* max(X), 0, 1.3 * max(Y)]) #Set the axes range.
    plt.title("Best line.")
    plt.text(10, 130, "m="+str(round(m,4))+"  b="+str(round(b,4)) + " error="+str(compute_total_error(m,b)))
    plt.show() #shows the graph.
    return

In [19]:
def main(): #main driver function
    init_points = np.array([0.0,1.0]) #intial points
    newton_points = compute_newton(init_points, max_iter = 100) #find the solution
    print(newton_points)
    print("b = {0}, m = {1}, error = {2}".format(newton_points[1], newton_points[0], compute_total_error(newton_points[0], newton_points[1])))
    time_t = time.time() - time_t #end time
    plot_line_data(newton_points[0], newton_points[1]) #plot the line generated
    return

if __name__=='__main__':
    main()

TypeError: unhashable type